In [ ]:
import pandas as pd

from fuzzywuzzy import fuzz
from math import radians, sin, cos, sqrt, atan2

### Trataremos de unificar los restaurantes que aparezcan en ambas plataformas.

In [3]:
# Especifica la ruta del archivo JSON
ruta_archivo = 'metadata_FL_Google.json'
# Lee el archivo JSON de google y carga los datos en un DataFrame
df_metadada_google = pd.read_json(ruta_archivo, encoding='utf-8')


# Especifica la ruta del archivo JSON
ruta_archivo = 'yelp_business_fl.json'
# Lee el archivo JSON de yelp y carga los datos en un DataFrame
df_metadada_yelp = pd.read_json(ruta_archivo, encoding='utf-8')

In [4]:
# Eliminamos los restaurantes que pudieran estar duplicados en la plataforma google.
df_metadada_google = df_metadada_google.drop_duplicates(subset=["gmap_id"])
# Eliminamos las columnas que no seran de ayuda para este analisis.
df_metadada_google = df_metadada_google.drop(['category', 'avg_rating', 'num_of_reviews', 'MISC', 'state'], axis=1)
# Ordenamos las columnas para mejor entendimiento visual de los datos.
df_metadada_google = df_metadada_google[['gmap_id', 'name', 'latitude', 'longitude', 'address', 'estado']]
# Transformamos las columnas de geolocalizacion a booleanos.
df_metadada_google['latitude'] = pd.to_numeric(df_metadada_google['latitude'], errors='coerce')
df_metadada_google['longitude'] = pd.to_numeric(df_metadada_google['longitude'], errors='coerce')


# Eliminamos los restaurantes que pudieran estar duplicados en la plataforma yelp.
df_metadada_yelp = df_metadada_yelp.drop_duplicates(subset=["business_id"])
# Eliminamos las columnas que no seran de ayuda para este analisis.
df_metadada_yelp = df_metadada_yelp.drop(['stars', 'review_count', 'attributes', 'categories'], axis=1)
# Ordenamos las columnas para mejor entendimiento visual de los datos.
df_metadada_yelp  = df_metadada_yelp [['business_id', 'name', 'latitude', 'longitude', 'address', 'city', 'state']]
# Transformamos las columnas de geolocalizacion a booleanos.
df_metadada_yelp['latitude'] = pd.to_numeric(df_metadada_yelp['latitude'], errors='coerce')
df_metadada_yelp['longitude'] = pd.to_numeric(df_metadada_yelp['longitude'], errors='coerce')

In [5]:
# Revisamos las columnas de  dataframe de google.
df_metadada_google.head()

,gmap_id,name,latitude,longitude,address,estado
0,0x88db4147b1d9e6f3:0x943dbd10a92ba1b1,Cape Seafood Shack,26.641377,-81.940545,"Cape Seafood Shack, 603 Del Prado Blvd S, Cape...",FL
1,0x88c2e4e34f1ed783:0x76c5da381c499d79,Fresh Point Country Buffet,27.867489,-82.702971,"Fresh Point Country Buffet, 10525 US-19, Pinel...",FL
2,0x88dae191ee505917:0x6ba3e25388d3fad4,Oneyda's Bakery,26.154754,-81.790528,"Oneyda's Bakery, 600 Goodlette-Frank Rd #101, ...",FL
3,0x88d9c754413f6c9d:0x1f93eff5e0ba9c16,Annie's Bake Shoppe,25.717416,-80.361945,"Annie's Bake Shoppe, 10331 SW 54th St, Miami, ...",FL
4,0x88d9b99475d9fd7b:0xea6083d207b2471a,Tropical Park Liquors,25.733914,-80.325216,"Tropical Park Liquors, 7971 SW 40th St Suite #...",FL


In [6]:
# # Revisamos las columnas de  dataframe de yelp.
df_metadada_yelp.head()

,business_id,name,latitude,longitude,address,city,state
0,0bPLkL0QhhPO5kt1_EXmNQ,Zio's Italian Market,27.916116,-82.760461,2575 E Bay Dr,Largo,FL
1,cVBxfMC4lp3DnocjYA3FHQ,Paws The Cat Cafe,53.549633,-113.508780,10588 109 Street,Edmonton,FL
2,seKihQKpGGnCeLuELRQPSQ,Twin Peaks,39.906295,-86.047463,6880 E 82nd St,Indianapolis,FL
3,L_TT0BFmFwORAMaA83K54A,Village Tap Room,39.869911,-86.143577,838 Broad Ripple Ave,Indianapolis,FL
4,ppFCk9aQkM338Rgwpl2F5A,Wawa,39.954573,-75.194894,3604 Chestnut St,Philadelphia,FL


### Con la información organizada, comenzamos el proceso de encontrar y unificar restaurantes

In [ ]:
# Crear una lista para almacenar los resultados.
resultados_lista = []

# Establecer un rango máximo de distancia para considerar como "cercanos".
rango_maximo = 0.250  # En kilómetros, es decir, 250 metros.

# Iterar sobre cada comercio de google y compararlo con cada comercio de yelp.
for index1, row1 in df_metadada_google.iterrows():
    for index2, row2 in df_metadada_yelp.iterrows():
        lat1, lon1 = row1['latitude'], row1['longitude']
        lat2, lon2 = row2['latitude'], row2['longitude']

        # Calcular la distancia entre los dos comercios.
        R = 6371  # Radio aproximado de la Tierra en kilómetros.
        lat1, lon1, lat2, lon2 = map(radians, [lat1, lon1, lat2, lon2])
        dlat = lat2 - lat1
        dlon = lon2 - lon1
        a = sin(dlat / 2)**2 + cos(lat1) * cos(lat2) * sin(dlon / 2)**2
        c = 2 * atan2(sqrt(a), sqrt(1 - a))
        distancia = R * c

        # Si la distancia está dentro del rango establecido, considerarlos como cercanos.
        if distancia <= rango_maximo:
            # Si los locales son cercanos, analizar el nombre de los mismos.
            ratio_similitud = fuzz.ratio(row1['name'], row2['name'])
            # Si el nombre es lo suficiente simimilar se concidera que es el mismo comercio.
            if ratio_similitud > 80:  # Umbral de similitud.
                # Agregar resultados a la lista de resultados.
                resultados_lista.append({
                    'gmap_id': row1['gmap_id'],
                    'business_id': row2['business_id'],
                    'name_g': row1['name'],
                    'name_y': row2['name'],
                    'address_g': row1['address'],
                    'address_y': f"{row2['address']}, {row2['city']}"
                })


# Guardar el DataFrame de resultados en un archivo si es necesario
# resultados.to_csv('resultados.csv', index=False)

In [8]:
# Pasamos los resultados a un dataframe.
resultados = pd.DataFrame(resultados_lista)

In [9]:
# Las coincidencias que vemos son a modo de ejemplo, por la cantidad de registros el proceso puede llegar a durar horas.
resultados

,gmap_id,business_id,name_g,name_y,address_g,address_y
0,0x88c2f730debd585f:0xeb36db3a693c980f,iuBcrPk2AsXu7x98tO63cA,Mainstay Tavern,Mainstay Tavern,"Mainstay Tavern, Clearwater, FL 33767","1160 Gulf Blvd, Clearwater"
1,0x88c2c4f199418f8f:0x610540c2747e67a6,NYjJ9KZM4xxcYtDpjkTSkg,Bennigan's,Bennigan's,"Bennigan's, 615 Channelside Dr #5942, Tampa, F...","615 Channelside Dr, Suite 205, Tampa"
2,0x88c32349f7b7e54b:0xd8683bee55d578e,R_PDoxc8hKzl_5TVittHHA,Publix Liquors at Gateway Commons,Publix at Gateway Commons,"Publix Liquors at Gateway Commons, 9518 Buffal...","9520 Buffalo Rd, Palmetto"
3,0x88c2fd6652a25e5b:0x25effa6bd3446b62,BSJE3vxVO9Q3SQygTEmZtQ,Marker 14,Marker 14,"Marker 14, 400 Treasure Island Causeway, Treas...","400 Treasure Island Cswy, Treasure Island"
4,0x88c2c023ffffffff:0xb846ce4c5a3a97eb,5Vrbqr9DK1gkrHGAS7y5eg,Teavana,Teavana,"Teavana, 7961 Citrus Park Town Center Mall, Ta...","7961 Citrus Park Town Center Mall, Tampa"
5,0x88c2ce79331a2e25:0xfdc8416438f82bf7,QlR8UCQ2xvRcw-eYhbWXGw,Tia's Tex Mex,Tias Tex Mex,"Tia's Tex Mex, 1414 W Brandon Blvd, Brandon, F...","1414 W Brandon Blvd, Brandon"
